In [1]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import sklearn
import sklearn.metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold
import optuna.integration.lightgbm as lgb

In [2]:
all_new = pd.read_csv('data/all_nlp_preprocessed.csv')
bert_embeded = pd.read_csv('kaggle-notebook/data/bert_embeded_pretrained_cat2.csv')
tfidf_500 = pd.read_csv('data/tfidf_500.csv')
tfidf_1000 = pd.read_csv('data/tfidf_1000.csv')
tfidf_2000 = pd.read_csv('data/tfidf_2000.csv')
mfw_df = pd.read_csv('data/mfw.csv')
miw_df = pd.read_csv('data/miw.csv')
sub = pd.read_csv('data/sample_submit.csv')

In [3]:
all_new = pd.merge(all_new, bert_embeded, on="id", how='outer')
all_new = pd.merge(all_new, tfidf_1000, on="id", how="outer")
#all_new = pd.merge(all_new, mfw_df, on="id", how="outer")
all_new = pd.merge(all_new, miw_df, on="id", how="outer")

In [4]:
all_new[[f"{n+50}" for n in range(25)]] = np.array(all_new[[f"{n}" for n in range(25)]]) + np.array(all_new[[f"{n+25}" for n in range(25)]])

In [5]:
def make_tag_set(df):
    tags = set()
    for compiled_html_text in df.html_compiled:
        tags |= set(extract_html_tag(compiled_html_text))
    return tags


def extract_html_tag(html_text):
    tag_list = re.findall(r"<.*?>", html_text)
    return tag_list


def remove_endtag(tagset):
    taglist = list(tagset)
    tags = set()
    for tag in taglist:
        if tag != "</animate>":
            tag = tag.replace("</", "<")
        tags.add(tag)
    return tags

def get_tag_collumns(tagset):
    columns = []
    for tag in tagset:
        columns.append(f"number_of_{tag}")
    return columns
    
ALL_TAGS = make_tag_set(all_new)
ALL_TAGS = remove_endtag(ALL_TAGS)

In [6]:
def make_pair(candidates):
    pairs = []
    N = len(candidates)
    for n, first in enumerate(candidates):
        for k in range(n+1,N):
            second = candidates[k]
            pairs.append([first, second])
    return pairs


def get_div_columns(pairs):
    columns = []
    for pair in pairs:
        columnA, columnB = pair
        columns.append(f'{columnA.split("_")[-1]}_per_{columnB.split("_")[-1]}')
    return columns

candidates = ["goal_min",
                  "number_of_word",
                  "duration",
                  "number_of_sentence",
                  "number_of_<p>",
                  "number_of_<figure>",]
div_tuples = make_pair(candidates)

In [7]:
TARGET_COLS = ["state"]
CATEGORICAL_COLS = ['country', "category1"]
categ1_cols = [f"{n+25}" for n in range(25)]
categ2_cols = [f"{n}" for n in range(25)]
categ1_2_cols = [f"{n+50}" for n in range(25)]
bert_cols = [f"bert_{n}" for n in range(768)]
tfidf_cols = tfidf_1000.columns.drop("id")
mfw_cols = [f"wordvec_{n}" for n in range(25)]
tag_cols = get_tag_collumns(ALL_TAGS)
div_cols = get_div_columns(div_tuples)

FEATURE_COLS = [
 'country',
 'duration',
 'category1',
 'goal_min',
 'number_of_word',
 'number_of_sentence',
]

FEATURE_COLS.extend(categ2_cols)
FEATURE_COLS.extend(categ1_2_cols)
#FEATURE_COLS.extend(bert_cols)
FEATURE_COLS.extend(tag_cols)
FEATURE_COLS.extend(div_cols)
FEATURE_COLS.extend(tfidf_cols)
FEATURE_COLS.extend(mfw_cols)
print(f"{6} + {len(categ2_cols)} + {len(categ1_2_cols)} + {len(tag_cols)} + {len(div_cols)} + {len(tfidf_cols)} + {len(mfw_cols)} = {len(FEATURE_COLS)}")

params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
    }

6 + 25 + 25 + 32 + 15 + 1000 + 25 = 1128


In [8]:
test_set = all_new[all_new["data_type"] == "test"]
train_set = all_new[all_new["data_type"] == "train"]

In [23]:
def run_null_optuna_lgbm(train, target_cols, feature_cols, categorical_cols):
    """
    import optuna.integration.lightgbm as lgb
    """
    X_train = train[feature_cols]
    y_train = train[target_cols]
    #X_test = test[feature_cols]
    
    #y_preds = []
    #models = []
    #oof_train = np.zeros((len(X_train),))
    importances = []
    
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
        X_tr = X_train.loc[train_index, :]
        X_val = X_train.loc[valid_index, :]
        
        # null target
        y_tr = np.array(y_train.loc[train_index]).squeeze()
        y_val = np.array(y_train.loc[valid_index]).squeeze()
        random.shuffle(y_tr)
        random.shuffle(y_val)

        lgb_train = lgb.Dataset(X_tr,
                                y_tr,
                                categorical_feature=categorical_cols)

        lgb_eval = lgb.Dataset(X_val,
                               y_val,
                               reference=lgb_train,
                               categorical_feature=categorical_cols)
        
        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          verbose_eval=False,
                          num_boost_round=1000,
                          early_stopping_rounds=5,
                          )
        
        """
        oof_train[valid_index] = model.predict(X_val,
                                               num_iteration=model.best_iteration)
        y_pred = model.predict(X_test,
                               num_iteration=model.best_iteration)

        y_preds.append(y_pred)
        models.append(model)
        """
        
        # display importance
        importance = pd.DataFrame(model.feature_importance(), index=feature_cols, columns=['importance'])
        importance = importance.sort_values("importance")
        display(importance)
        importances.append(importance)

    #return oof_train, sum(y_preds) / len(y_preds), importances
    return importances

In [24]:
def check_null_importances(num_iteration=10):
    feature_cols = FEATURE_COLS
    categorical_cols = CATEGORICAL_COLS
    target_cols = TARGET_COLS
    
    null_importances = []
    for n in range(num_iteration):
        importances = run_null_optuna_lgbm(train_set, target_cols, feature_cols, categorical_cols)
        null_importances.extend(importances)
    return null_importances

In [26]:
null_importances = check_null_importances(num_iteration=1)

[I 2021-01-08 16:40:03,977] A new study created in memory with name: no-name-8786cb2f-9f44-44d7-9c13-aeab629d8256


  0%|          | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 4244, number of negative: 4192
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.039286 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 164185
[LightGBM] [Info] Number of data points in the train set: 8436, number of used features: 1127




feature_fraction, val_score: 0.693474:   0%|          | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: 0.693474:  14%|#4        | 1/7 [00:00<00:02,  2.05it/s][I 2021-01-08 16:40:04,472] Trial 0 finished with value: 0.6934736127804769 and parameters: {'feature_fraction': 0.4}. Best is trial 0 with value: 0.6934736127804769.


feature_fraction, val_score: 0.693474:  14%|#4        | 1/7 [00:00<00:02,  2.05it/s]

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503082 -> initscore=0.012328
[LightGBM] [Info] Start training from score 0.012328
[LightGBM] [Info] Number of positive: 4244, number of negative: 4192
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033030 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 164185
[LightGBM] [Info] Number of data points in the train set: 8436, number of used features: 1127




feature_fraction, val_score: 0.693383:  14%|#4        | 1/7 [00:00<00:02,  2.05it/s]

feature_fraction, val_score: 0.693383:  29%|##8       | 2/7 [00:00<00:02,  2.02it/s][I 2021-01-08 16:40:04,983] Trial 1 finished with value: 0.6933833587904045 and parameters: {'feature_fraction': 0.5}. Best is trial 1 with value: 0.6933833587904045.


feature_fraction, val_score: 0.693383:  29%|##8       | 2/7 [00:01<00:02,  2.02it/s]

...

min_data_in_leaf, val_score: 0.692752:  80%|########  | 4/5 [00:02<00:00,  1.46it/s]

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506638 -> initscore=0.026554
[LightGBM] [Info] Start training from score 0.026554
[LightGBM] [Info] Number of positive: 4274, number of negative: 4162
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050514 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 164613
[LightGBM] [Info] Number of data points in the train set: 8436, number of used features: 1126


min_data_in_leaf, val_score: 0.692752: 100%|##########| 5/5 [00:03<00:00,  1.46it/s]

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506638 -> initscore=0.026554
[LightGBM] [Info] Start training from score 0.026554


,importance
country,0
tfidf_pen,0
tfidf_people,0
tfidf_perfect,0
tfidf_perform,0
...,...
tfidf_don,2
word_per_<p>,2
tfidf_home,2
23,2


In [45]:
for null_importance in null_importances:
    null_index = null_importance[null_importance["importance"] > 0].index
    print(null_index)

Index(['tfidf_et', 'tfidf_write', 'tfidf_equipment', 'wordvec_16',
       'wordvec_17', '9', '51', 'tfidf_price', 'tfidf_cover', 'wordvec_20',
       'duration', 'wordvec_22', 'tfidf_culture', 'tfidf_money',
       'tfidf_worked', 'tfidf_hope', 'tfidf_creative', 'wordvec_5',
       'tfidf_started', 'tfidf_images', '13', 'tfidf_specific', 'tfidf_york',
       'wordvec_10', 'wordvec_15', '1', '20', 'wordvec_2', 'wordvec_1',
       'number_of_word', 'tfidf_huge', 'tfidf_written', '19',
       'tfidf_professional', 'tfidf_open', 'tfidf_woman', 'tfidf_society',
       'tfidf_fine', 'tfidf_bit', 'tfidf_sales', 'tfidf_dream',
       'tfidf_understand', 'tfidf_local', 'tfidf_photo', 'tfidf_game',
       'tfidf_personal', 'min_per_sentence', 'tfidf_california', 'tfidf_15',
       'tfidf_looking', 'tfidf_passion', 'tfidf_make', 'duration_per_<p>',
       'tfidf_love', 'tfidf_distribution', 'tfidf_thank', 'tfidf_goals',
       'tfidf_pictures', 'tfidf_challenge', 'tfidf_reading', 'tfidf_added',
 